In [248]:
from sympy import *
import numpy as np
import scipy as sp
from scipy.linalg import lu
from scipy.sparse.linalg import spilu
import math
import sympy 
import itertools
import sys
import time
import multiprocessing

from __future__ import print_function


from copy import deepcopy
import time
import fractions
import timeit

%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


# print function

In [414]:
def print_row(row,symbols,nvar,cal=False):
    # nvar  
    plus_flag = False
    
    for ele in row:
        mul_flag = False
        if ele[0] == 0:
            continue
        constant_flag = True
        for i in range(nvar+1):
            if i==0:
                if plus_flag and ele[i]>0:
                    print('+',end ='')
                if ele[i] != 1:
                    if cal:
                        print(ele[i],end="")
                    else:
                        print("%2.2f"%ele[i],end="")
                    mul_flag = True
                plus_flag = True
                continue
            if ele[i] != 0:
                if mul_flag:
                    print('*',end = "")
                if cal :
                    mul_flag = True
                print(symbols[i-1],end = "")
                if ele[i] > 1:
                    if cal:
                        print("**",end="")
                    else:
                        print("^",end="")
                    print(ele[i],end="")
                constant_flag = False
        
        if constant_flag and ele[0]==1:
            if cal:
                print(ele[0],end="")
            else:
                print("%2.2f"%ele[0],end="")
        plus_flag = True
    if cal:
        print(",")
    else:
        print("")
            
def print_all(M,nvar,symbols=[],cal=False):
    if len(symbols) == 0:
        symbols = ['x0','x1','x2','x3','x4','x5','x6','x7'][:nvar]
    for row in M:
        if len(row)!=0:
            print_row(row,symbols,nvar,cal)

## Generate Gauissian Quaduature 

In [250]:
# nomal Gaussian Quaduature
def GQ(n=2):
    result = []
    for i in range(2*n):
        p = []
        for j in range(n):
            p_s = np.zeros(2*n+1,dtype = int)
            p_s[0] = 1
            p_s[2*j+1] = 1
            p_s[2*j+2] = i
            p.append(p_s.tolist())
        if i-i/2*2 == 0:
            p_s = np.zeros(2*n+1)
            p_s[0] = -2./(i+1)
            p.append(p_s.tolist())
        result.append(p)
    return result

def GQ_sym(n=2):
    result = []
    for i in range(2*n):
        if i-i/2*2 == 0:
            c = "A%d"%(i/2)
            result.append(c)
        else:
            c = "x%d"%(i/2)
            result.append(c)
    return result

In [251]:
def GQS(n=2):
    odd = False
    if n-n/2*2 == 1:
        odd = True
    result = []
    if True:
        for i in range(2*n):
            p = []
            for j in range(n/2):
                p_s = np.zeros(2*n - n/2*2+1,dtype = int)
                p_s[0] = 1
                p_s[2*j+1] = 1
                p_s[2*j+2] = i
                p.append(p_s.tolist())
                
                p_s = np.zeros(2*n - n/2*2+1,dtype = int)
                p_s[0] = 1
                if i-i/2*2 >0:
                    p_s[0] = -1
                p_s[2*j+1] = 1
                p_s[2*j+2] = i
                p.append(p_s.tolist())
            if odd:
                p_s = np.zeros(2*n - n/2*2+1,dtype = int)
                p_s[0] = 1
                p_s[2*n - n/2*2-1] = 1
                p_s[2*n - n/2*2] = i
                p.append(p_s.tolist())
            
            if i-i/2*2 == 0 :
                p_s = np.zeros(2*n - n/2*2+1)
                p_s[0] = -2./(i+1)
                p.append(p_s.tolist())
            
            result.append(p)
        return result
    for i in range(2*n):
        p = []
        for j in range(n/2):
            p_s = np.zeros(2*n+1,dtype = int)
            p_s[0] = 1
            p_s[2*j+1] = 1
            p_s[2*j+2] = i             
            p.append(p_s.tolist())
                
            p_s = np.zeros(2*n+1,dtype = int)
            p_s[0] = 1
            if i-i/2*2 >0:
                p_s[0] = -1
            p_s[2*j+1] = 1
            p_s[2*j+2] = i
            p.append(p_s.tolist())
        if i-i/2*2 == 0:
            p_s = np.zeros(n+1)
            p_s[0] = -2./(i+1)
            p.append(p_s.tolist())
        result.append(p)
    return result

# formation of the coefficient matri for multi-variable

In [252]:
def matC_sparse(polynomials, mdeg,nvar=2):
    row = []
    col =[]
    ele = []
    
    rows = len(polynomials)
    cols = mdeg**nvar
    
    for i in range(rows):
        for monomial in polynomials[i]:
            s = cols-1
            for j in range(nvar):
                s -= monomial[j+1]*mdeg**(nvar-1-j)
            row.append(i)
            col.append(s)
            ele.append(1.0*monomial[0])
    M = sp.sparse.csr_matrix((ele, (row, col)), shape=(rows, cols))
    return M


# Gaussian Elimination

In [253]:
def GEPP_sparse(C):    
    M1 = C.copy()
    M1 = M1.tocsc()
    n = M1.shape[0]  ## number of rows
    m = M1.shape[1]  ## number of columns

    currentrow = 0
    for i in range(m):
        if M1[currentrow:,i].nnz==0:
            pass  ## do nothing if we have a column of zeros
        else:
            if currentrow < (n-1):
                    #print(i)
                    ## first find index of (abs) maximum value
                non_zero_row = M1[currentrow:n,i]
                sub_index = np.argmax(abs(sp.sparse.find(non_zero_row)[2]))
                index = sp.sparse.find(non_zero_row)[0][sub_index]+currentrow  #### n\n-1 
                    #print(M[currentrow:n,i])
                    #print(index)
                    #print("index is", index,"\n")
                # swap rows
                a_idx = np.where(M1.indices == index)
                b_idx = np.where(M1.indices == currentrow)
                M1.indices[a_idx] = currentrow
                M1.indices[b_idx] = index    
                          ####
                #print("the matrix is now \n",M,"\n")
                #print(M)

                    ## running the elimination step
                for j in range(currentrow+1,n):
                    if M1[j,i]!=0:
                            #print("currentrow column entry is", M[currentrow,i],"\n")
                        multiplier = M1[j,i]/M1[currentrow,i]
                            #print("multiplier is", multiplier,"\n")
                        M1[j,i:] = M1[j,i:] - multiplier*M1[currentrow,i:]
                    else:
                        pass
                    #print("the matrix is after elimination \n",M,"\n")

                    ## incrementing row so the algorithm stops when we have a triangular system
    
                currentrow = currentrow + 1
    
    M1=M1.tocsr()
    
    for j in range(n):
        if M1[j,:].nnz==0:
            continue  ## do nothing if we have a row of zeros
        else:
       

            indptr = M1.indptr
            a = indptr[j]
            b = indptr[j+1]
            M1.data[a:b] = M1.data[a:b]/M1.data[a] 
      

    return M1.tocsr()


# Buchberger's algorithm for multi-variable

In [254]:
## sparse

# input the start th leading
def leading_sparse(row,start=0):
    if row.nnz==0:
        return -1
    if start >= row.nnz:
        return -1
        
        print("leading term overflow")
        print(row)
        print(start)
        sys.exit()
    
    return row.indices[start]
    # return sp.sparse.find(row)[1][start]
    

def leading_term_sparse(arg,mdeg,nvar):
    # return leading term without coefficient
    
    result = np.zeros(nvar)
    a = arg
    b = 0
    for i in range(nvar):
        div = mdeg**(nvar-1-i)
        b = a - a/div*div
        a = a/div
        result[i]=mdeg-1-a
        a = b
    return result

def check_multiple_sparse(lead_1,lead_2,nvar):
    # check lead_1 is a multiple of lead_2
    for i in range(nvar):
        if lead_1[i]<lead_2[i]:
            return False
    return True

def shift_col_sparse(lcm,deg_i,mdeg,nvar):
    a = lcm[:-1]-deg_i
    shift_column = 0
    for i in range(nvar):
        shift_column += mdeg**(nvar-1-i)*a[i]
    return int(shift_column)

# input sparse matirx
def shift_sparse(lcm,deg_i,mdeg,nvar,row=-1,leading=-1,S=(-1,-1)):
    row = row.tocsr()
    row_copy = deepcopy(row)
    shift_column = shift_col_sparse(lcm,deg_i,mdeg,nvar)
    if leading-shift_column < 0:
        print(S,leading,shift_column)
        print(row)
        print("degree overflow")
        sys.exit()
    row_copy.indices[:] -= shift_column

    return row_copy

def get_lcm_sparse(a,b): 
    return abs(a * b) / fractions.gcd(a,b) if a and b else 0

def check_multiple_sparse(lead_1,lead_2,nvar):
    # check lead_1 is a multiple of lead_2
    for i in range(nvar):
        if lead_1[i]<lead_2[i]:
            return False
    return True

def remainder_sparse(M,new_row,mdeg, nvar):
    M = M.tocsr()
    flag = True
    m = M.shape[0]
    while flag:
        flag = False
        new_lead = leading_sparse(new_row)
        if new_lead==-1:
            break
        new_lead_term = leading_term_sparse(new_lead,mdeg,nvar)
        for i in range(m):
            row_lead = leading_sparse(M[i])
            row_lead_term = leading_term_sparse(row_lead,mdeg,nvar)
            if check_multiple_sparse(new_lead_term,row_lead_term,nvar):
                #print(i)
                lcm = np.hstack((new_lead_term,[1]))
                old_shifted = shift_col_sparse(lcm,row_lead_term,mdeg,nvar)
                coeff = new_row[0,new_lead]
                
                current_row = deepcopy(M[i])
                #print(current_row)
                current_row.indices[:] -= old_shifted
                #print(current_row)
                
                new_row -= coeff * current_row
                flag = True
                break
    return new_row

def Buchberger_sparse(M, mdeg, nvar, S):
    # check leading term
    a = M[S[0]]
    b = M[S[1]]
    
    arg_a = leading_sparse(a)
    arg_b = leading_sparse(b)
    
    if arg_a==-1 or arg_b==-1:
        return 0,M
        
        print("0 row occur")
        print(S)
        print(M)
        sys.exit()
    
    deg_a = leading_term_sparse(arg_a,mdeg,nvar)
    deg_b = leading_term_sparse(arg_b,mdeg,nvar)
    
    
    lcm = np.zeros(nvar+1)
    for i in range(nvar):
        lcm[i] = max(deg_a[i],deg_b[i])
        
    if(a[0,arg_a]<0):
        a *= -1
        
    if(b[0,arg_b]<0):
        b *= -1

    lcm[nvar] = get_lcm_sparse(a[0,arg_a],b[0,arg_b])
    

    a_shifted = shift_sparse(lcm,deg_a,mdeg,nvar,a,arg_a,S)
    b_shifted = shift_sparse(lcm,deg_b,mdeg,nvar,b,arg_b,S)
    
    """   
    if a[0,arg_a]!=1 or b[0,arg_b]!=1:
        print("Normalization in Gaussian Elimination Failed")
        print(M)
        sys.exit()
    """
    if a[0,arg_a]> b[0,arg_b]:
        new_row = a_shifted - (a[0,arg_a]/ b[0,arg_b])*b_shifted
    else:
        new_row = (b[0,arg_b]/a[0,arg_a])*a_shifted - b_shifted
    
    new_row = remainder_sparse(M,new_row,mdeg, nvar)
    

    if new_row.nnz == 0:
        return 0,M
    
    # normalize new column
    arg_c = leading_sparse(new_row)
    
    if new_row[0,arg_c]!=1:
        new_row.data[:] /= new_row[0,arg_c]
    M = sp.sparse.vstack([M,new_row])
    return 1,M




# Reduced 

In [255]:
def find_largest(lead_v):
    # l is the largest
    l = -1
    arg = -1
    
    m = lead_v.shape[0]
    
    for row in range(m):
        if l == -1:
            l = lead_v[row,-1]
            arg = row
            continue
        if lead_v[row,-1] != -1 and lead_v[row,-1]<l:
            l = lead_v[row,-1]
            arg = row
    if l == -1:
        return -1
    return arg

def renew_lead_v(M,mdeg,nvar,lead_v,row,starts,tol=10**-4):
    m = M.shape[0]
    
    lead_v[row,-1] = leading_sparse(M[row],starts[row])
    while abs(M[row,lead_v[row,-1]]) < tol:
        
        M[row,lead_v[row,-1]]=0
        M.eliminate_zeros()
        lead_v[row,-1] = leading_sparse(M[row],starts[row])

        if lead_v[row,-1] == -1:
            return M,lead_v    
    if lead_v[row,-1] == -1:
        return M,lead_v   
    monomial = leading_term_sparse(lead_v[row,-1],mdeg,nvar)
    for j in range(nvar):
        lead_v[row,j] = monomial[j]
    return M,lead_v
    
def reduced_form(M,mdeg,nvar,tol):
    m,n = M.shape
    
    # the vector store the leading term
    lead_v = np.zeros((m,nvar+1),dtype = int)
    
    # vector stores the i the leading term
    starts = np.zeros(m,dtype = int)
    
    # initialization leading_vector
    for row in range(m):
        M,lead_v = renew_lead_v(M,mdeg,nvar,lead_v,row,starts,tol)
    
    large_row = find_largest(lead_v)
    while large_row != -1:
        flag = False
        for row in range(m):
            if row != large_row and starts[row] == 0:
                lead_arg = leading_sparse(M[row])
                lead_term = leading_term_sparse(lead_arg,mdeg,nvar)
                if check_multiple_sparse(lead_v[large_row,:-1],lead_term,nvar):

                    shift_col = shift_col_sparse(lead_v[large_row],lead_term,mdeg,nvar)
                    indices = deepcopy(M[row].indices) - shift_col
                    indptr = np.zeros(m+1,dtype = int)
                    for i in range(large_row+1,m+1):
                        indptr[i] = indices.shape[0]
                    data = deepcopy(M[row].data)
                    M_subtrac = sp.sparse.csr_matrix((data, indices, indptr), shape=(m, n))
                    c = M[large_row,lead_v[large_row,-1]]/M[row,lead_v[row,-1]]
                    M -= c*M_subtrac
                    flag = True
                    break
        if flag == False:
            starts[large_row]+=1

        M,lead_v=renew_lead_v(M,mdeg,nvar,lead_v,large_row,starts,tol)
        large_row = find_largest(lead_v)
    
    return M
    #print_all(matrix_to_array_sparse(M,mdeg,nvar),['A0','x0','A1','x1'],nvar)

    

# Combine them together

In [256]:
def delete_row_csr(mat, i):
    if not isinstance(mat, sp.sparse.csr_matrix):
        raise ValueError("works only for CSR format -- use .tocsr() first")
    n = mat.indptr[i+1] - mat.indptr[i]
    if n > 0:
        mat.data[mat.indptr[i]:-n] = mat.data[mat.indptr[i+1]:]
        mat.data = mat.data[:-n]
        mat.indices[mat.indptr[i]:-n] = mat.indices[mat.indptr[i+1]:]
        mat.indices = mat.indices[:-n]
    mat.indptr[i:-1] = mat.indptr[i+1:]
    mat.indptr[i:] -= n
    mat.indptr = mat.indptr[:-1]
    mat._shape = (mat._shape[0]-1, mat._shape[1])

def trun_zero_sparse(M):
    while M[-1,:].nnz==0:
        M = M[:-1,:]
    return M

def normalize_leading_sparse(M):
    m = M.shape[0]
    for i in range(m):
        lead = leading_sparse(M[i])
        if lead == -1:
            #delete_row_csr(M, i)
            continue
        indptr = M.indptr
        a = indptr[i]
        b = indptr[i+1]
        M.data[a:b] = M.data[a:b]/M.data[a]
    return M

In [257]:
def check_nvar(row):
    ele = 0
    arg = -1
    nvar = row.shape[0]
    for i in range(nvar):
        if row[i] == 1:
            ele += 1
            arg = i
    if ele == 1:
        return arg
    if ele == 0:
        return -2
    return -1

def check_terminate(M,mdeg,nvar):
    M.eliminate_zeros()
    m = M.shape[0]
    
    Var = np.zeros((m,nvar))
    for i in range(m):
        for j in range(M[i].nnz):
            lead = leading_sparse(M[i],j)
            if lead == -1:
                print("check_terminate problem")
                sys.exit()
            term = leading_term_sparse(lead,mdeg,nvar)
            for k in range(nvar):
                if term[k] != 0:
                    Var[i,k] = 1.
    C_E = np.zeros(nvar,dtype = int)
    flag = True
    
    #print(Var)
    while(flag):
        flag = False
        row_num = 0
        while row_num<Var.shape[0]:
            c = check_nvar(Var[row_num])
            #print(row_num,c)
            #print(Var)
            if c > -1:
                C_E[c] = 1
                Var[:,c] = 0
                Var=np.delete(Var, row_num, 0)
                flag = True
                break
            if c == -2:
                Var=np.delete(Var, row_num, 0)
                continue
            row_num += 1
    #print(C_E)
    if sum(C_E) == nvar:
        return True
    return False

In [269]:
def run_sparse(M, mdeg, nvar,tol = 10**-14,details = False,cal = False,Gro = False):
    #M = GEPP_sparse(M)
    
    max_row = M.shape[0]-1
    current_row = 0
    flag = -1
    M=reduced_form(M,mdeg,nvar,tol)
    if details:
        print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(nvar/2),cal)
        print("")
    #M=normalize_leading_sparse(M)
    while current_row != max_row:
        #print(current_row,max_row)
        current_row +=1
        #M = GEPP(M)
        for i in range(current_row):
            M=normalize_leading_sparse(M)
            flag,M = Buchberger_sparse(M, mdeg, nvar, [current_row,i])
            if flag == 1:
                M=reduced_form(M,mdeg,nvar,tol)
                if details:
                    print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(nvar/2),cal)
                    print("")
                if Gro == False:
                    if check_terminate(M,mdeg,nvar):
                        return M
        #time.sleep(1)
        #M = GEPP(M)
        #M = trun_zero(M)
        M=normalize_leading_sparse(M)
        #print(M)
        max_row = M.shape[0]-1
    return M


In [270]:
def matrix_to_array_sparse(M,mdeg,nvar):
    array = []
    m = M.shape[0]
    for i in range(m):
        array.append([])
        index = 0
        nnz = M[i].nnz
        for j in range(nnz):
            index = leading_sparse(M[i],j)
            L_T = leading_term_sparse(index,mdeg,nvar)
            add = [M[i,index]]
            add.extend(L_T)
            array[i].append(add)

    return array

def print_array_sparse(array):
    for p in array:
        print(p)

# Testing

Here we test $p_1=x^2$ and $p_2=xy+y^2$. And get Grobner basis $g_1=x^2, g_2=xy+y^2, g_3 = y^3$.

In [271]:
p1 = [(1,2,0)]
p2 = [(1,1,1),(1,0,2)]
polynomials = [p1,p2]
mdeg = 5
nvar = 2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar)
#M.toarray()
print_all(matrix_to_array_sparse(M,mdeg,nvar),2,['x','y']) 


x^2.0
xy+y^2.0
y^3.0


Here we test $p_1=xyz+xy$ and $p_2=x^2y^2z+yz$. And get Grobner basis $g_1=x^2y^2z+yz, g_2=xyz+xy, g_3=x^2y^2-2yz, g_4 = yz^2+yz$.

In [272]:
p1 = [(1,1,1,1),(1,1,1,0)]
p2 = [(2,0,1,1),(1,2,2,1)]
polynomials = [p1,p2]
mdeg = 5
nvar = 3
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar)
#M.toarray()
print_all(matrix_to_array_sparse(M,mdeg,nvar),3,['x','y','z']) 

xyz+xy
x^2.0y^2.0-2.00*y*z
yz^2.0+yz


# Solving Gaussian Quaduature for $n=2$

In [273]:
p1= [(1,1,0,0,0),(1,0,0,1,0),(-2,0,0,0,0)]
p2= [(1,1,1,0,0),(1,0,0,1,1)]
p3= [(1,1,2,0,0),(1,0,0,1,2),(-2.0/3,0,0,0,0)]
p4= [(1,1,3,0,0),(1,0,0,1,3)]
polynomials = [p1,p2,p3,p4]
mdeg = 10
nvar = 4
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,['A0','x0','A1','x1'])

A0-1.00
-1.00*x1^2.0+0.33
-0.33*x0-0.33*x1
0.33*A1+x1^2.0-0.67


In [274]:
%memit matC_sparse(polynomials,mdeg,nvar)

peak memory: 70.50 MiB, increment: 0.00 MiB


In [275]:
M = matC_sparse(polynomials,mdeg,nvar)

def T(M,mdeg,nvar,loop=10):
    for i in range(loop):
        M = matC_sparse(polynomials,mdeg,nvar)
        run_sparse(M, mdeg,nvar)
%memit run_sparse(M, mdeg,nvar)

peak memory: 70.50 MiB, increment: 0.00 MiB


In [276]:
M = matC_sparse(polynomials,mdeg,nvar)
%timeit run_sparse(M, mdeg,nvar) 

10 loops, best of 3: 25.2 ms per loop


# Symmetric Gaussian Quaduature

## n = 3

In [277]:
n = 3
polynomials = GQS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = 2*n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = True,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 

print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n),cal = True)

2.0*A0-1.11111111111,
0.888888888889*x1,
-0.6*A1+0.533333333333,
0.666666666667*x0**2.0-0.4,

A0-0.555555555556,
x1,
A1-0.888888888889,
x0**2.0-0.6,


In [278]:
%%timeit
n = 3
polynomials = GQS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = 2*n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()

10 loops, best of 3: 32.6 ms per loop


In [279]:
%%memit
n = 3
polynomials = GQS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = 2*n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)

peak memory: 70.51 MiB, increment: 0.00 MiB


## n = 4

In [280]:
n = 4
polynomials = GQS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = 2*n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = True,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 

print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n),cal = True)

2.0*A0+2.0*A1-2.0,
-2.0*x0**2.0*A1+2.0*x0**2.0+2.0*A1*x1**2.0-0.666666666667,
-2.0*x0**2.0*x1**2.0+0.666666666667*x0**2.0+0.666666666667*x1**2.0-0.4,
-2.0*x0**4.0*x1**2.0+0.666666666667*x0**4.0+0.4*x1**2.0-0.285714285714,

A0+A1-1.0,
x0**2.0*A1+0.6*A1+x1**2.0-0.990476190476,
-0.444444444444*x1**4.0+0.380952380952*x1**2.0-0.0380952380952,
-0.2*x0**2.0-0.333333333333*x1**4.0+0.0857142857143*x1**2.0+0.142857142857,
0.190476190476*A1*x1**2.0+0.114285714286*A1-0.0888888888889,

A0+A1-1.0,
x0**2.0*A1+0.6*A1+x1**2.0-0.990476190476,
-0.444444444444*x1**4.0+0.380952380952*x1**2.0-0.0380952380952,
-0.2*x0**2.0-0.333333333333*x1**4.0+0.0857142857143*x1**2.0+0.142857142857,
0.190476190476*A1*x1**2.0+0.114285714286*A1-0.0888888888889,


In [281]:
%%timeit
n = 4
polynomials = GQS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = 2*n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)

10 loops, best of 3: 48.1 ms per loop


In [282]:
%%memit
n = 4
polynomials = GQS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = 2*n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)

peak memory: 70.51 MiB, increment: 0.00 MiB


In [283]:
A0=0.6521451548625461
x0=-0.3399810435848563
A1=0.3478548451374538
x1=0.8611363115940526
print(A0+A1-1.0,
x0**2.0*A1+0.6*A1+x1**2.0-0.990476190476,
-0.444444444444*x1**4.0+0.380952380952*x1**2.0-0.0380952380952,
-0.2*x0**2.0-0.333333333333*x1**4.0+0.0857142857143*x1**2.0+0.142857142857,
0.190476190476*A1*x1**2.0+0.114285714286*A1-0.0888888888889)

-1.11022302463e-16 1.90403248723e-13 -6.93889390391e-18 5.10425035571e-14 3.91214838302e-14


# n = 5

In [284]:
n = 5
polynomials = GQS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = 2*n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = True,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n),cal = True)



2.0*A0+2.0*A1+A2-2.0,
A2*x2,
-2.0*x0**2.0*A1-1.0*x0**2.0*A2+2.0*x0**2.0+2.0*A1*x1**2.0-0.666666666667,
x0**2.0*x1**2.0*A2-2.0*x0**2.0*x1**2.0+0.666666666667*x0**2.0+0.666666666667*x1**2.0-0.4,
x0**4.0*x1**2.0*A2-2.0*x0**4.0*x1**2.0+0.666666666667*x0**4.0+0.4*x1**2.0-0.285714285714,
x0**6.0*x1**2.0*A2-2.0*x0**6.0*x1**2.0+0.666666666667*x0**6.0+0.285714285714*x1**2.0-0.222222222222,

A0+A1-0.715555555556,
0.568888888889*x2,
x0**2.0*A1+0.230769230769*A1+0.715555555556*x1**2.0-0.710959164292,
0.238095238095*A2-0.13544973545,
0.106666666667*x1**4.0-0.118518518519*x1**2.0+0.0253968253968,
0.285714285714*x0**2.0+0.4*x1**4.0-0.15873015873*x1**2.0-0.222222222222,
-0.256790123457*A1*x1**2.0-0.0592592592593*A1-0.2*A2+0.177777777778,

A0+A1-0.715555555556,
0.568888888889*x2,
x0**2.0*A1+0.230769230769*A1+0.715555555556*x1**2.0-0.710959164292,
0.238095238095*A2-0.13544973545,
0.106666666667*x1**4.0-0.118518518519*x1**2.0+0.0253968253968,
0.285714285714*x0**2.0+0.4*x1**4.0-0.15873015873*x1**2.0-0.222

In [285]:
%%timeit
n = 5
polynomials = GQS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = 2*n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)

10 loops, best of 3: 98 ms per loop


In [286]:
%%memit
n = 5
polynomials = GQS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = 2*n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)

peak memory: 70.51 MiB, increment: 0.00 MiB


In [287]:
A2=0.5688888888888889
x2=0.0000000000000000
A1=0.4786286704993665
x1=-0.5384693101056831
A0=0.2369268850561891
x0=-0.9061798459386640
print(A0+A1-0.715555555556,
0.568888888889*x2,
x0**2.0*A1+0.230769230769*A1+0.715555555556*x1**2.0-0.710959164292,
0.238095238095*A2-0.13544973545,
0.106666666667*x1**4.0-0.118518518519*x1**2.0+0.0253968253968,
0.285714285714*x0**2.0+0.4*x1**4.0-0.15873015873*x1**2.0-0.222222222222,
-0.256790123457*A1*x1**2.0-0.0592592592593*A1-0.2*A2+0.177777777778)

-4.44422276757e-13 0.0 5.16031661846e-13 -4.00013355772e-13 -1.3697723511e-13 3.35842464949e-14 1.735833699e-13


# Solving Gaussian Quaduature without symmetry

## n = 3

In [288]:
n = 3
polynomials = GQ(n)
mdeg = 20
nvar = n*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,GQ_sym(n),cal = True)

A0+A1-0.555555555556*x2**2.0-1.11111111111,
0.45*A1*x2+0.9*x1*A2+0.25*x1*x2**2.0-0.8*x1-0.4*x2,
x2**3.0-0.6*x2,
-1.0*x1**2.0-1.0*x1*x2-1.0*x2**2.0+0.6,
x0+x1+1.66666666667*x2**3.0,
0.675*A1*A2-1.26666666667*A1-2.0*x1**2.0+0.75*x1*A2**2.0*x2-4.25*x1*A2*x2+0.625*x1*x2+0.675*A2**2.0-0.75462962963*x2**2.0+1.03703703704,
0.666666666667*A1*x1+0.333333333333*A1*x2+0.708333333333*x1*A2+0.208333333333*x1*x2**2.0-1.0*x1-0.37037037037*x2**3.0-0.0740740740741*x2,
-0.6*A2-0.333333333333*x2**2.0+0.533333333333,


In [289]:
A0=0.8888888888888888
x0=0.0000000000000000
A1=0.5555555555555556
x1=-0.7745966692414834
A2=0.5555555555555556
x2=0.7745966692414834

print(A0+A1-0.555555555556*x2**2.0-1.11111111111,
0.45*A1*x2+0.9*x1*A2+0.25*x1*x2**2.0-0.8*x1-0.4*x2,
x2**3.0-0.6*x2,
-1.0*x1**2.0-1.0*x1*x2-1.0*x2**2.0+0.6,
x0+x1+1.66666666667*x2**3.0,
0.675*A1*A2-1.26666666667*A1-2.0*x1**2.0+0.75*x1*A2**2.0*x2-4.25*x1*A2*x2+0.625*x1*x2+0.675*A2**2.0-0.75462962963*x2**2.0+1.03703703704,
0.666666666667*A1*x1+0.333333333333*A1*x2+0.708333333333*x1*A2+0.208333333333*x1*x2**2.0-1.0*x1-0.37037037037*x2**3.0-0.0740740740741*x2,
-0.6*A2-0.333333333333*x2**2.0+0.533333333333)

8.44213587925e-13 0.0 5.55111512313e-17 -1.11022302463e-16 1.54920520856e-12 8.88844553515e-13 1.63570545997e-13 -1.33337785257e-13


In [290]:
%%memit
n = 3
polynomials = GQ(n)
mdeg = 20
nvar = n*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)

peak memory: 70.51 MiB, increment: 0.00 MiB


In [291]:
%%timeit
n = 3
polynomials = GQ(n)
mdeg = 20
nvar = n*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)

1 loop, best of 3: 181 ms per loop


# Random polynomial

In [376]:
def ran(bound):
    a = int(np.random.random()*bound)
    return a

def Random_P(nvar = 2,ma = 2, m = 2, n =2 , coe = 4):
    Polys = []
    for i in range(m):
        poly = []
        for j in range(n):
            term = []
            term.append(1.*ran(coe)+1)
            for k in range(nvar):
                term.append(1.*ran(ma))
            poly.append(term)
        Polys.append(poly)
    return Polys
            

In [ ]:
n = 3
ma = 3
nvar = 3
m=3
mdeg = 100
coe = 4
polynomials = Random_P(nvar,ma, m , n  , coe)
M=matC_sparse(polynomials,mdeg,nvar)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar)

In [377]:
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar)

x0^2.0x1^2.0x2^2.0+3.00*x0*x1^2.0*x2^2.0+x1x2^2.0
x0^2.0x1x2+x0^2.0x2^2.0+4.00*x1
6.00*x0*x1*x2+3.00*x1

x0^2.0x2^2.0
-0.02*x1
x1^2.0+0.05*x1
x0x1+176.00*x1^2.0+13.00*x1


In [406]:
n = 3
ma = 3
nvar = 3
m=3
mdeg = 100
coe = 4
polynomials = Random_P(nvar,ma, m , n  , 1)
print(polynomials)
M=matC_sparse(polynomials,mdeg,nvar)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,['x','y','z'])
#print(M)
print("")
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,['x','y','z'],cal =True)

[[[1.0, 2.0, 0.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 0.0]], [[1.0, 2.0, 1.0, 0.0], [1.0, 1.0, 1.0, 0.0], [1.0, 2.0, 2.0, 2.0]], [[1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 0.0, 2.0], [1.0, 0.0, 0.0, 1.0]]]
x^2.0z+xyz+xy
x^2.0y^2.0z^2.0+x^2.0y+xy
xyz+xz^2.0+z

x*y-1.0*y*z**2.0-1.0*y*z+1.0*z,
-1.0*y**2.0*z**2.0-1.0*y**2.0*z-1.0*y*z**2.0-1.0*y*z+1.0*z,
x*z+1.0*y*z**2.0+1.0*y*z,
1.0*z**2.0-1.0*z,
y**3.0*z**2.0+y**3.0*z-1.0*y**2.0*z**2.0-1.0*y**2.0*z-3.0*y*z**2.0-2.0*y*z+7.0*z**2.0-5.0*z,
-1.5*y*z**3.0+1.5*y*z-1.5*z**2.0+1.5*z,


In [427]:
polynomials = [[[1.0, 2.0, 0.0, 1.0], [1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 0.0]],
               [[1.0, 2.0, 1.0, 0.0], [1.0, 1.0, 1.0, 0.0], [1.0, 2.0, 2.0, 2.0]],
               [[1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 0.0, 2.0], [1.0, 0.0, 0.0, 1.0]],
              [[1.0,0,0.,1.]]]

In [428]:
M=matC_sparse(polynomials,mdeg,nvar)
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,['x','y','z'],cal = True)
#print(M)
print("")
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,['x','y','z'],cal =True)

x**2.0*z+x*y*z+x*y,
x**2.0*y**2.0*z**2.0+x**2.0*y+x*y,
x*y*z+x*z**2.0+z,
z,

x*y,
x**2.0*y+x*y,
z,


In [419]:
z = 1.
y = -.5+.5*np.sqrt(3)
x = 2.*y**2-1

In [420]:
print(x**2.0*z+x*y*z+x*y,
x**2.0*y**2.0*z**2.0+x**2.0*y+x*y,
x*y*z+x*z**2.0+z,
z-1.0)

2.22044604925e-16 1.11022302463e-16 0.0 0.0


In [421]:
z = 1.
y = -.5-.5*np.sqrt(3)
x = 2.*y**2-1

In [422]:
print(x**2.0*z+x*y*z+x*y,
x**2.0*y**2.0*z**2.0+x**2.0*y+x*y,
x*y*z+x*z**2.0+z,
z-1.0)

0.0 -4.4408920985e-16 0.0 0.0


In [432]:
z = 0.
y = 10.
x = 0
print(x**2.0*z+x*y*z+x*y,
x**2.0*y**2.0*z**2.0+x**2.0*y+x*y,
x*y*z+x*z**2.0+z)

0.0 0.0 0.0


# Constant Weight

In [461]:
def CWS(n=2):
    odd = False
    if n-n/2*2 == 1:
        odd = True
    result = []
    w = 2./n
    
    if True:
        for i in range(n+1):
            p = []
            if i == 0:
                continue
            for j in range(n/2):
                p_s = np.zeros(n/2+n - n/2*2+1)
                p_s[0] = w
                p_s[j+1] = i
                p.append(p_s.tolist())
                
                p_s = np.zeros(n/2+n - n/2*2+1)
                p_s[0] = w
                if i-i/2*2 >0:
                    p_s[0] = -w
                p_s[j+1] = i
                p.append(p_s.tolist())
            if odd:
                p_s = np.zeros(n/2+n - n/2*2+1)
                p_s[0] = w
                p_s[n/2+n - n/2*2] = i
                p.append(p_s.tolist())
            
            if i-i/2*2 == 0 :
                p_s = np.zeros(n/2+n - n/2*2+1)
                p_s[0] = -2./(i+1)
                p.append(p_s.tolist())
            
            result.append(p)
        return result


## n = 2

In [466]:
n = 2
polynomials = CWS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = n/2+n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,cal = True)

x0**2.0-0.333333333333,


## n = 3

In [468]:
n = 3
polynomials = CWS(n)
#print_array_sparse(polynomials)
mdeg = 20
nvar = n/2+n-n/2*2
M=matC_sparse(polynomials,mdeg,nvar)
#print(M)
M = run_sparse(M, mdeg,nvar,tol=10**-5,details = False,cal=True)
#M.toarray()
#print_array_sparse(matrix_to_array_sparse(M,mdeg,nvar)) 
print_all(matrix_to_array_sparse(M,mdeg,nvar),nvar,cal = True)

x1,
x0**2.0-0.5,
